In [1]:
import pandas as pd
import learning_spoons_quants as quant
import matplotlib.pyplot as plt

In [2]:
finance_df = quant.finance_data_preprocessing(r'재무데이터 수정.xlsx')
price_df = quant.price_data_preprocs(r'수정주가.csv')

In [3]:
finance_df.head()

Symbol Name   ROA                                                \
        Symbol Name 08/4Q 09/1Q 09/2Q 09/3Q  09/4Q  10/1Q  10/2Q  10/3Q   
Symbol                                                                    
A005930        삼성전자  7.62  5.24  3.78  5.46   6.93   8.74  10.53  11.46   
A000660      SK하이닉스     0     0     0     0      0  12.57  18.73  25.35   
A068270        셀트리온  2.92  4.89  8.63   7.3  10.48  11.75  14.02  17.22   
A207940    삼성바이오로직스     0     0     0     0      0      0      0      0   
A005380         현대차   4.5  3.94  4.67   6.6   8.36  10.01     10   9.82   

                ...    DPS                                                   \
         10/4Q  ...  16/1Q  16/2Q  16/3Q  16/4Q  17/1Q  17/2Q  17/3Q  17/4Q   
Symbol          ...                                                           
A005930   12.1  ...    570    570    570    570    850    850    850    850   
A000660  17.41  ...    600    600    600    600   1000   1000   1000   1000   
A068270   9.68  ...  46.73  46.73  46.73  46.73  19.61  19.61  19.61  19.61   
A207940      0  ...      0      0      0      0      0      0      0      0   
A005380   7.82  ...   4000   4000   4000   4000   4000   4000   4000   4000   

                       
         18/1Q  18/2Q  
Symbol                 
A005930    850    850  
A000660   1000   1000  
A068270  19.61  19.61  
A207940      0      0  
A005380   4000   4000  

[5 rows x 650 columns]

In [4]:
price_df.head()

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A900340,A950010,A950030,A950070,A950100,A950110,A950130,A950140,A950160,A950170
코드명,,,,,,,,,,,,,,,,,,,,,
2006-01-31,NaN,3430,NaN,2317,12164,5788,57432,NaN,NaN,120947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-28,NaN,3440,NaN,2279,9693,5015,55082,NaN,NaN,120947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-03-31,NaN,3080,NaN,2039,10187,4775,58556,NaN,NaN,111188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-04-28,NaN,5040,NaN,1918,10985,6060,62133,NaN,NaN,108050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-05-30,NaN,5500,NaN,1332,10643,6008,57432,NaN,NaN,101079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

start_date = '2016-1'
initial_money = 100000000
riskfree_rate = 0.02
rebal_period = 6
num_of_rebal = 3
st_function = quant.Fama_LSV    

backtest_price = price_df[start_date: ]

In [6]:
for num in range(0, num_of_rebal):
    
    if num == 0:
        initial_money = initial_money
        temp_date = start_date
    else:
        initial_money = total_backtest['종합포트'][-1]
        temp_date = str(total_backtest.index[-1])
        
    year = temp_date.split('-')[0]
    month = temp_date.split('-')[1]
    
    if month in ['1', '2', '3']:
        st_date = str(int(year) - 1)[2:] + '/3Q'
    elif month in ['4', '5', '6']:
        st_date = str(int(year) - 1)[2:] + '/4Q'
    elif month in ['7', '8', '9']:
        st_date = year[2:] + '/1Q'
    else:
        st_date = year[2:] + '/2Q'

    temp_index = backtest_price.iloc[rebal_period * num].dropna().index   
    
    st_df = st_function(finance_df.loc[temp_index], st_date, 10)  
    
    selected_list = st_df.index   
    
    print(list( st_df['Symbol Name']['Symbol Name']))
        
    each_money = initial_money / len(selected_list)

    pf_value = 0
    for code in selected_list:
        temp_num = int( each_money / int( backtest_price.iloc[rebal_period * num][code] ) ) 
        pf_value = pf_value + pd.to_numeric(backtest_price[code][rebal_period * num : rebal_period * (num + 1) + 1]) * temp_num

    cash = initial_money - pf_value[0]
    cash_pf = []
    for i in range(0, len(pf_value)):
        cash_pf.append( cash * (1 + riskfree_rate/12) ** i )

    backtest_df = pd.DataFrame( {'주식포트': pf_value} )
    backtest_df['현금포트'] = cash_pf
    backtest_df['종합포트'] = backtest_df['주식포트'] + backtest_df['현금포트']
    backtest_df['월별수익률'] = backtest_df['종합포트'].pct_change() * 100
    
    if num == 0 :
        backtest_df['누적수익률'] = (backtest_df['종합포트'] / backtest_df['종합포트'][0] - 1) * 100
        total_backtest = backtest_df
    else:
        backtest_df['누적수익률'] = (backtest_df['종합포트'] / total_backtest['종합포트'][0] - 1) * 100
        total_backtest = pd.concat([total_backtest, backtest_df[1:]])

C:\Users\skii4\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


['정다운', '인화정공', '동일제강', '오스템', 'DSR제강', '경남스틸', '일정실업', '쎄노텍', '제일테크노스', '유성티엔에스']
['케이프', '고려시멘트', '이노인스트루먼트', '쎄노텍', '씨아이에스', '성도이엔지', 'DSR제강', '한국석유', '우신시스템', '동방']
['RFHIC', '차이나하오란', '글로벌텍스프리', '인화정공', '그랜드백화점', '로보로보', '클래시스', '이엑스티', '하이스틸', '러셀']


In [7]:
total_backtest

,주식포트,현금포트,종합포트,월별수익률,누적수익률
코드명,,,,,
2016-01-29,99960728,39272.000000,1.000000e+08,NaN,0.000000
2016-02-29,99650146,39337.453333,9.968948e+07,-0.310517,-0.310517
2016-03-31,107754390,39403.015756,1.077938e+08,8.129553,7.793793
2016-04-29,112693360,39468.687448,1.127328e+08,4.581930,12.732829
2016-05-31,118503440,39534.468594,1.185430e+08,5.153908,18.542974
2016-06-30,119666282,39600.359375,1.197059e+08,0.981001,19.705882
2016-07-29,131702800,39666.359974,1.317425e+08,10.055132,31.742466
2016-08-31,131892596,85784.095574,1.319784e+08,0.179072,31.978380
2016-09-30,135071525,85927.069067,1.351575e+08,2.408782,35.157452


# 함수화 작업

In [9]:
def backtest(start_date, initial_money, riskfree_rate, 
             rebal_period, num_of_rebal, st_function, pfnum, price_df, finance_df):

    backtest_price = price_df[start_date: ]

    for num in range(0, num_of_rebal):

        if num == 0:
            initial_money = initial_money
            temp_date = start_date
        else:
            initial_money = total_backtest['종합포트'][-1]
            temp_date = str(total_backtest.index[-1])

        year = temp_date.split('-')[0]
        month = temp_date.split('-')[1]

        if month in ['1', '2', '3']:
            st_date = str(int(year) - 1)[2:] + '/3Q'
        elif month in ['4', '5', '6']:
            st_date = str(int(year) - 1)[2:] + '/4Q'
        elif month in ['7', '8', '9']:
            st_date = year[2:] + '/1Q'
        else:
            st_date = year[2:] + '/2Q'

        temp_index = backtest_price.iloc[rebal_period * num].dropna().index   

        st_df = st_function(finance_df.loc[temp_index], st_date, pfnum)  

        selected_list = st_df.index   

        print(list( st_df['Symbol Name']['Symbol Name']))

        each_money = initial_money / len(selected_list)

        pf_value = 0
        for code in selected_list:
            temp_num = int( each_money / int( backtest_price.iloc[rebal_period * num][code] ) ) 
            pf_value = pf_value + pd.to_numeric(backtest_price[code][rebal_period * num : rebal_period * (num + 1) + 1]) * temp_num

        cash = initial_money - pf_value[0]
        cash_pf = []
        for i in range(0, len(pf_value)):
            cash_pf.append( cash * (1 + riskfree_rate/12) ** i )

        backtest_df = pd.DataFrame( {'주식포트': pf_value} )
        backtest_df['현금포트'] = cash_pf
        backtest_df['종합포트'] = backtest_df['주식포트'] + backtest_df['현금포트']
        backtest_df['월별수익률'] = backtest_df['종합포트'].pct_change() * 100
        if num == 0 :
            backtest_df['누적수익률'] = (backtest_df['종합포트'] / backtest_df['종합포트'][0] - 1) * 100
            total_backtest = backtest_df
        else:
            backtest_df['누적수익률'] = (backtest_df['종합포트'] / total_backtest['종합포트'][0] - 1) * 100
            total_backtest = pd.concat([total_backtest, backtest_df[1:]])
    return total_backtest

In [10]:
finance_df = quant.finance_data_preprocessing(r'재무데이터 수정.xlsx')
price_df = quant.price_data_preprocs(r'수정주가.csv')

In [11]:
start_date = '2016-1'
initial_money = 100000000
riskfree_rate = 0.02
rebal_period = 6
num_of_rebal = 3
st_function = quant.NCAV 
pfnum = 20

backtest(start_date, initial_money, riskfree_rate, 
             rebal_period, num_of_rebal, st_function, pfnum, price_df, finance_df)

C:\Users\skii4\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


['심텍홀딩스', '휴젤', '다우기술', '엠케이전자', '다우데이타', 'KISCO홀딩스', '차이나그레이트', 'S&T홀딩스', '더블유게임즈', '동부제철', '크리스탈신소재', '잇츠한불', '코리아오토글라스', '아이디스홀딩스', '케어젠', '연우', '서연', 'SG&G', '성도이엔지', '뉴트리바이오텍']
['현대건설', '두산', '서연', 'KISCO홀딩스', '다우기술', '차이나그레이트', '세원정공', 'KPX홀딩스', 'KG케미칼', '일동홀딩스', '다우데이타', '씨아이에스', '이노인스트루먼트', 'SG&G', '성도이엔지', '차이나하오란', '세보엠이씨', '아이디스홀딩스', 'S&T홀딩스', 'CS홀딩스']
['다우기술', 'KISCO홀딩스', '두산', 'SJM홀딩스', 'S&T홀딩스', '리드코프', '아이디스홀딩스', 'KPX홀딩스', '매일홀딩스', '에스앤씨엔진그룹', 'RFHIC', 'KTcs', '만호제강', '클래시스', '러셀', '이스트아시아홀딩스', '이엑스티', 'CS홀딩스', '다우데이타', 'KG케미칼']


,주식포트,현금포트,종합포트,월별수익률,누적수익률
코드명,,,,,
2016-01-29,99656134,343866.000000,1.000000e+08,NaN,0.000000
2016-02-29,95552885,344439.110000,9.589732e+07,-4.102676,-4.102676
2016-03-31,101715584,345013.175183,1.020606e+08,6.426950,2.060597
2016-04-29,104467503,345588.197142,1.048131e+08,2.696921,4.813091
2016-05-31,107228144,346164.177471,1.075743e+08,2.634420,7.574308
2016-06-30,106306050,346741.117766,1.066528e+08,-0.856633,6.652791
2016-07-29,109429525,347319.019629,1.097768e+08,2.929181,9.776844
2016-08-31,112820528,256542.879662,1.130771e+08,3.006305,13.077071
2016-09-30,109542156,256970.451128,1.097991e+08,-2.898859,9.799126
